In [279]:
# with open('../gsk-ene-1.1-bccwj-json-jumanpp-type/bccwj-ene-jumanpp-type.txt') as f:
#     sentences = f.read().split('\n\n')
#     sentences = [[line.split(' ') for line in sentence.split('\n') if len(line.split(' '))==5] for sentence in sentences]
#     pos = set([line[1] for sentence in sentences for line in sentence])
#     bunrui = set([line[2] for sentence in sentences for line in sentence])
#     chunk = set([line[3] for sentence in sentences for line in sentence])
# pos, bunrui, chunk

In [282]:
# for p in sorted(bunrui):
#     print(f'FinePOSLabelMapper.add("{p}")')

In [280]:
# typomap = {'B-Food': 'Dish',
#   'B-Freguency': 'B-Frequency',
#   'B-Offense': 'B-Offence',
#   'B-Period_time': 'B-Period_Time',
#   'B-Position_vocation': 'B-Position_Vocation',
#   'I-Food': 'I-Dish',
#   'I-Freguency': 'I-Frequency',
#   'I-Offense': 'I-Offence',
#   'I-Period_time': 'I-Period_Time',
#   'I-Position_vocation': 'I-Position_Vocation',
#           }

# with open('labels_enesub.txt') as f:
#     LABELS = {l for l in f.read().split('\n') if l}
# print(len(LABELS))

# labels = set()
# from collections import Counter
# with open('../gsk-ene-1.1-bccwj-json-jumanpp-type/bccwj-ene-jumanpp-type.txt') as f,\
#      open('../gsk-ene-1.1-bccwj-json-jumanpp-type/bccwj-ene-jumanpp-type-sub-fix.txt', 'w') as of:
#     sentences = f.read().split('\n\n')
#     sentences = [[line.split(' ') for line in sentence.split('\n') if len(line.split(' '))==5] for sentence in sentences]
#     for sentence in sentences:
#         if len(sentence) > 1:
#             if sentence[-1][0] != '。':
#                 sentence.append(['。','特殊', '句点', 'O', 'O'])

#             for line in sentence:
#                 if line[-1] not in LABELS:
#                     line[-1] = 'O'
#                 if line[-1] in typomap:
#                     line[-1] = typomap[line[-1]]
#                 labels.add(line[-1])
#                 of.write(' '.join(line))
#                 of.write('\n')
#             of.write('\n')
#     lasts = Counter([sentence[-1][0] for sentence in sentences])
# print(lasts, len(labels), len(LABELS))


In [281]:
# with open('labels_enesub.txt') as f:
#     LABELS = [l for l in f.read().split('\n') if l]
# print(len(LABELS))
# with open('ene-labels-bccwj.txt', 'w') as f:
#     for l in LABELS:
#         if l in labels:
#             f.write(l)
#             f.write('\n')

In [14]:
def read_classification_report(filename):
    with open(filename) as f:
        s = f.read()
    lines = [ss.strip().split('     ') for ss in s.split('\n') if ss]
    micro = []
    macro = []
    wmicro = []
    for i, line in enumerate(lines):
        if i % 3 == 0:
            micro.append({'p': line[1], 'r': line[2], 'f1': line[3], 's': line[4]})
        elif i % 3 == 1:
            macro.append({'p': line[1], 'r': line[2], 'f1': line[3], 's': line[4]})
        else:
            wmicro.append({'p': line[1], 'r': line[2], 'f1': line[3], 's': line[4]})
    return {'micro': micro, 'macro': macro, 'wmicro': wmicro}
# read_classification_report('classification_base_ep9.txt')
# read_classification_report('classification_kb_ohe_ep9.txt')
# read_classification_report('classification_kb_ce_ep9.txt')

In [98]:
import random

def select(x, k=10):
    return random.sample(x, k)

def select_dict(x, k=10):
    return sorted(x.items(), key=lambda x: -x[1])[:10]

In [137]:
from sklearn.metrics import confusion_matrix
def show_confusion(y_true, y_pred, LABELS, min_count=0):
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)  # axis=(gold, pred)
    print('gold -> pred : support')
    for gold_i, a in enumerate(cm):
        for pred_i, c in enumerate(a):
            if gold_i != pred_i and c > min_count:
                print(f'{LABELS[gold_i]} -> {LABELS[pred_i]} : {c}')


y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
LABELS = ["ant", "bird", "cat"]
print(list(enumerate(LABELS)))
show_confusion(y_true, y_pred, LABELS, min_count=0)

[(0, 'ant'), (1, 'bird'), (2, 'cat')]
gold -> pred : support
bird -> cat : 1
cat -> ant : 1


In [189]:
from collections import Counter

def read_token_label_list(filename):
    with open(filename) as f:
        sentences = [[l.split('\t') for l in s.split('\n') if len(l.split('\t')) == 3] for s in f.read().split('\n\n')]
    print(len(sentences))
    sentences_err = [s for s in sentences if any(pred != gold for surf, pred, gold in s)]
    print(len(sentences_err))
    sentences_pos = [s for s in sentences if all(pred == gold for surf, pred, gold in s)]
    return sentences_err, sentences_pos

def categorize_errors(sentences_err):
    fns = []
    fps_o = []
    fps_conf = []

    for i, s in enumerate(sentences_err):
        sent = '\n'.join( ' '.join(line) for line in s)
    #     print(sent)
        for surf, pred, gold in s:
            if pred == 'O' and gold != 'O':  # gold != 'O'; FN(ひろいこぼし)
                fns.append({'target':(surf, pred, gold), 'sentence': sent , 'sid': i})
            elif pred != 'O':  # FP
                if gold == 'O':
                    fps_o.append({'target':(surf, pred, gold), 'sentence': sent , 'sid': i})
                elif pred != gold:  # gold != 'O'
                    fps_conf.append({'target':(surf, pred, gold), 'sentence': sent , 'sid': i})
    print(len(sentences_err), len(fns), len(fps_o), len(fps_conf))
    return fns, fps_o, fps_conf


def categorize_errors_detail(fns, fps_conf, fps_o):

    # FN
    fns_counter = Counter(d['target'][2].split('-')[1] for d in fns)
    fns_entries = {d['target'][2].split('-')[1]: [] for d in fns}
    fns_uniq = {d['sid']: [] for d in fns}
    for d in fns:
        fns_uniq[ d['sid'] ].append(d['target'])
        fns_entries[ d['target'][2].split('-')[1] ].append(d['target'])

    # FP: confusion
    fps_conf_g_counter = Counter(d['target'][2].split('-')[1] for d in fps_conf)
    fps_conf_g_entries = {d['target'][2].split('-')[1]: [] for d in fps_conf}
    fps_conf_p_counter = Counter(d['target'][1].split('-')[1] for d in fps_conf if d['target'][1] != 'X')
    fps_conf_p_entries = {d['target'][1].split('-')[1]: [] for d in fps_conf if d['target'][1] != 'X'}
    fps_conf_uniq = {d['sid']: [] for d in fps_conf}

    y_gold, y_pred = [], []
    labels = set()
    fps_confusion = {(d['target'][2], d['target'][1]): [] for d in fps_conf}

    for d in fps_conf:
        fps_conf_uniq[ d['sid'] ].append(d['target'])
        fps_conf_g_entries[ d['target'][2].split('-')[1] ].append(d['target'])
        if d['target'][1] != 'X':
            fps_conf_p_entries[ d['target'][1].split('-')[1] ].append(d['target'])
            y_gold.append(d['target'][2])
            y_pred.append(d['target'][1])
            labels.add(d['target'][2])
            labels.add(d['target'][1])
            fps_confusion[(d['target'][2], d['target'][1])].append(d['target'])
    labels = sorted(labels, key=lambda x: x.split('-')[1]+x[0].split('-')[0])

    # FP: O as positive
    fps_o_counter = Counter(d['target'][1] for d in fps_o)  # .split('-')[1]
    fps_o_entries = {d['target'][1]: [] for d in fps_o}  # .split('-')[1]
    fps_o_uniq = {d['sid']: [] for d in fps_o}
    for d in fps_o:
        fps_o_uniq[ d['sid'] ].append(d['target'])
        fps_o_entries[ d['target'][1] ].append(d['target'])  # .split('-')[1]

    return {
'fn': {
'sid2result': fns_uniq,
'counter_gold': fns_counter,
'entry_map_gold': fns_entries,
},

'fp_confusion': {
'sid2result': fps_conf_uniq,
'counter_gold': fps_conf_g_counter, 'counter_pred': fps_conf_p_counter,
'entry_map_gold': fps_conf_g_entries, 'entry_map_pred': fps_conf_p_entries,
'confusion_info': {'y_gold': y_gold, 'y_pred': y_pred, 'labels': labels, 'confusion_map': fps_confusion}
},
    
'fp_o': {
'sid2result': fps_o_uniq,
'counter_pred': fps_o_counter,
'entry_map_pred': fps_o_entries,
}
    }


def error_analysis(filename):
    sentences_err, sentences_pos = read_token_label_list(filename)
    fns, fps_o, fps_conf = categorize_errors(sentences_err)
    return categorize_errors_detail(fns, fps_conf, fps_o)


In [190]:

error_detail = error_analysis('output_result_base_ep9/token_label_list_epoch9_greedy_fix.txt')


3369
1566
1566 2217 1578 1120


In [211]:
# 未抽出誤り; 知識依存
fn = error_detail['fn']
len(fn['sid2result']), select_dict(fn['counter_gold'])

(886,
 [('Position_Vocation', 566),
  ('Person', 289),
  ('Company', 161),
  ('Corporation_Other', 143),
  ('Government', 118),
  ('Organization_Other', 112),
  ('GOE_Other', 107),
  ('City', 102),
  ('International_Organization', 84),
  ('Country', 57)])

In [212]:
# FP混同; 文脈・語彙の問題?; KB的に対処可能
# - 正解との部分一致: 処理が面倒（一致するところも保持）
# - クラス誤り

fp_confusion = error_detail['fp_confusion']
len(fp_confusion['sid2result']), select_dict(fp_confusion['counter_gold']), select_dict(fp_confusion['counter_pred'])
#x[0].split('-')[1]+x[0].split('-')[0])

(637,
 [('Position_Vocation', 257),
  ('Person', 106),
  ('Company', 83),
  ('Government', 79),
  ('City', 67),
  ('GOE_Other', 53),
  ('Corporation_Other', 51),
  ('Organization_Other', 47),
  ('Public_Institution', 31),
  ('Sports_Organization_Other', 30)],
 [('Position_Vocation', 224),
  ('Person', 163),
  ('GOE_Other', 85),
  ('Corporation_Other', 84),
  ('Government', 84),
  ('City', 80),
  ('Company', 74),
  ('Public_Institution', 37),
  ('Country', 36),
  ('Province', 32)])

In [213]:
# FP; 過抽出誤り; 文脈・語彙の問題?

fp_o = error_detail['fp_o']
len(fp_o['sid2result']), select_dict(fp_o['counter_pred'])
#x[0].split('-')[1]+x[0].split('-')[0])

(799,
 [('B-Position_Vocation', 244),
  ('B-Person', 213),
  ('I-Position_Vocation', 182),
  ('X', 177),
  ('I-Person', 76),
  ('B-City', 69),
  ('I-GOE_Other', 59),
  ('B-Country', 57),
  ('B-Company', 56),
  ('B-GOE_Other', 40)])

In [214]:
# FN errors
sum([v for k, v in fn['counter_gold'].items()]), sum(v for k, v in fp_confusion['counter_gold'].items())

(2217, 1120)

In [215]:
# FP error
sum([v for k, v in fp_confusion['counter_pred'].items()]), sum(v for k, v in fp_o['counter_pred'].items())

(1105, 1578)

In [220]:
from sklearn_crfsuite import metrics

metrics.flat_classification_report

<function sklearn_crfsuite.metrics.flat_classification_report(y_true, y_pred, labels=None, **kwargs)>

# FN: 未抽出誤り
- 知識ベースで改善する見込みがある

In [ ]:
select(list(fn['entry_map_gold'].items()))

In [193]:
select(fn['entry_map_gold']['Person'])

[('[UNK]', 'O', 'B-Person'),
 ('パウンド', 'O', 'B-Person'),
 ('ガトームソン', 'O', 'B-Person'),
 ('Ｐ', 'O', 'I-Person'),
 ('竹蔵', 'O', 'B-Person'),
 ('美紗', 'O', 'B-Person'),
 ('\u3000', 'O', 'I-Person'),
 ('トルシエ', 'O', 'B-Person'),
 ('サザエ', 'O', 'B-Person'),
 ('文公', 'O', 'B-Person')]

# FP混同
## - 正解との部分一致
## - クラス誤り

In [ ]:
select(list(fp_confusion['entry_map_gold'].items()))

In [196]:
select(fp_confusion['entry_map_gold']['Person'])

[('武寿', 'B-City', 'B-Person'),
 ('家宝', 'B-Person', 'I-Person'),
 ('赤松', 'B-Family', 'B-Person'),
 ('シー', 'I-Position_Vocation', 'B-Person'),
 ('新太', 'B-Person', 'I-Person'),
 ('教夫', 'B-Person', 'I-Person'),
 ('深町', 'B-City', 'B-Person'),
 ('高崎', 'B-Position_Vocation', 'B-Person'),
 ('新渡戸', 'B-Station', 'B-Person'),
 ('信雄', 'B-Person', 'I-Person')]

In [197]:
# 'confusion_info': {'y_gold': y_gold, 'y_pred': y_pred, 'labels': labels, 'confusion_map': fps_confusion}
confusion_info = fp_confusion['confusion_info']
y_gold, y_pred, labels = confusion_info['y_gold'], confusion_info['y_pred'], confusion_info['labels'], 

fps_confusion_s = {f'{k[0]} -> {k[1]}': v for k, v in confusion_info['confusion_map'].items()}

show_confusion(y_gold, y_pred, labels, min_count=5)

gold -> pred : support
B-City -> B-GOE_Other : 11
B-City -> B-Person : 14
B-City -> B-Province : 13
B-Company -> B-Corporation_Other : 7
B-Company -> B-GOE_Other : 9
B-Company -> B-Person : 9
I-Company -> B-Company : 6
I-Company -> I-GOE_Other : 9
B-Company_Group -> B-Company : 7
I-Corporation_Other -> B-Corporation_Other : 6
I-Corporation_Other -> I-Government : 18
B-Country -> B-Nationality : 6
B-Family -> B-Person : 6
B-GOE_Other -> B-City : 11
I-GOE_Other -> B-GOE_Other : 8
B-GPE_Other -> B-City : 6
B-Government -> B-Corporation_Other : 8
B-Government -> I-Government : 9
B-Government -> B-Position_Vocation : 10
I-Government -> I-Corporation_Other : 20
I-Government -> I-Position_Vocation : 6
I-Organization_Other -> I-Corporation_Other : 6
I-Organization_Other -> I-Government : 7
B-Person -> B-City : 13
B-Person -> B-Company : 10
B-Person -> I-Person : 9
B-Person -> B-Position_Vocation : 7
I-Person -> B-Person : 28
B-Position_Vocation -> B-Person : 20
B-Position_Vocation -> I-Person 

In [198]:
fps_confusion_s['B-Company -> B-GOE_Other']

[('吉野', 'B-GOE_Other', 'B-Company'),
 ('吉野', 'B-GOE_Other', 'B-Company'),
 ('吉野', 'B-GOE_Other', 'B-Company'),
 ('トゥモロー', 'B-GOE_Other', 'B-Company'),
 ('ジャパレン', 'B-GOE_Other', 'B-Company'),
 ('ＳＯＧＯ', 'B-GOE_Other', 'B-Company'),
 ('大丸', 'B-GOE_Other', 'B-Company'),
 ('飯塚', 'B-GOE_Other', 'B-Company'),
 ('ラヂオ', 'B-GOE_Other', 'B-Company')]

# FP; 過抽出誤り

In [ ]:
select(list(fp_o['entry_map_pred'].items()))